In [15]:
dataset_path = "<Dataset Folder Path Present on Drive>"

In [16]:
import numpy as np
import pandas as pd
import collections,os,json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.core.display import HTML,display
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=RuntimeWarning)
import pickle

C:\Users\ravin\AppData\Local\Temp\ipykernel_33420\4051890243.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML,display


In [17]:
def csspath():
  return display(HTML("""<style>
  #output-body {
    display: flex;
    align-items: center;
    justify-content: center;
    background-color:white;
    color:black;
    }
  .output_text pre{
    margin-right : 70px;
    color: black;
    font-weight: bold;
    /* font-size: 10px; */
  }

}  </style>"""))

#Data Exploration

##Data Processing

In [18]:
def dateprocessattributes(data):
  data["publish_time"] = pd.to_datetime(data["publish_time"])
  data["trending_date"] = pd.to_datetime(data["trending_date"],format="%y.%d.%m")
  data["trend_year"]=data["trending_date"].apply(lambda time:time.year)
  data["trend_date"] = data["trending_date"].apply(lambda time: time.day)
  data["trend_month"]=data["trending_date"].apply(lambda time:time.month)
  data["publish_year"]=data["publish_time"].apply(lambda time:time.year)
  data["publish_month"]=data["publish_time"].apply(lambda time:time.month)
  data["publish_day"]=data["publish_time"].apply(lambda time:time.day)
  data["publish_weekday"]=data["publish_time"].apply(lambda time:time.dayofweek)
  data["publish_hour"]=data["publish_time"].apply(lambda time:time.hour)
  return data

In [19]:
def categmatch(x,dictionary): #function to get the categories
    try:
        return dictionary[str(x)]
    except:
        return "Unknown"

##Data Cleaning

In [20]:
def cleandata(data):
  if data['description'].isnull().sum()>0:
    data['description'] = data['description'].replace(np.nan,"") #Replacing the NaN values in the "Description" column with an empty string
  data = data.sort_values('video_id').drop_duplicates(subset=['trending_date', 'title'], keep='last')
  data = data.drop(columns=["publish_time","category_id"],axis=1)
  return data

##Loading the datasets

In [21]:
def load_data(file_path): 
    chfile=file_path
    codecountry = chfile[0:2]
    path=os.path.join(dataset_path,chfile)
    inputdf=pd.read_csv(path,encoding='utf-8')
    titledict={}
    pathjson=os.path.join(dataset_path,(codecountry+str('_category_id.json')))
    jsoninputdf=pd.read_json(pathjson)
    inputdf['region']=codecountry
    print('HI')
    for dict_a in jsoninputdf['items']:
        titledict[dict_a['id']]=dict_a['snippet']['title'] 
    inputdf = dateprocessattributes(inputdf) #Calling the function to process the date columns.
    inputdf['category']=inputdf['category_id'].apply(categmatch,args=(titledict,))
    inputdf = cleandata(inputdf)
    return inputdf

In [22]:
csspath()
conlist = []
condict = {}
tlist = []
t_df = pd.DataFrame()
for dataset_pathname, _, filenames in os.walk(dataset_path):
    print("................................")
    for paths in filenames:
        if paths.lower().endswith(".csv"):
          conlist.append(paths[0:2])
          x = load_data(paths)
          tlist.append(x)
          x = x.drop(columns=['video_id','region'],axis=1)
          globals()[str(paths[0:2])] = x
          condict[str(paths[0:2])]=x
          print("------Created",paths[0:2],"DataFrame------")
    print("Processed all available Datasets")
t_df = pd.concat(tlist,ignore_index=True)
tdf = t_df
tdf.name = "The 6 Countries"
print("................................")
print("tdf  is the cumulative DataFrame")

................................
HI
------Created CA DataFrame------
HI
------Created DE DataFrame------
HI
------Created FR DataFrame------


C:\Users\ravin\AppData\Local\Temp\ipykernel_33420\1869322958.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  inputdf=pd.read_csv(path,encoding='utf-8')


HI
------Created GB DataFrame------
HI
------Created IN DataFrame------
HI
------Created US DataFrame------
Processed all available Datasets
................................
HI
------Created CA DataFrame------
HI
------Created DE DataFrame------
HI
------Created FR DataFrame------


C:\Users\ravin\AppData\Local\Temp\ipykernel_33420\1869322958.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  inputdf=pd.read_csv(path,encoding='utf-8')


HI
------Created GB DataFrame------
HI
------Created IN DataFrame------
Processed all available Datasets
................................
tdf  is the cumulative DataFrame


In [23]:
def getuniqcateg(data):
  x_cat = data['category'].unique()
  y_cat = list(x_cat)
  if "Unknown" in y_cat:
    y_cat.remove('Unknown')
  cdict = {}
  for i in range(0,len(y_cat)):
    cdict[i] = y_cat[i]
  return cdict,x_cat
# dict_cat,x_cat = getuniqcateg(tdf)

##Filling the Missing Categories using Word Vectorisers and Classification

In [24]:
for i,j in condict.items():
  print(i,j[j.category=="Unknown"].value_counts().sum())

CA 74
DE 256
FR 114
GB 90
IN 103
US 0


In [25]:
gnb_model = MultinomialNB()
rfc_model= RandomForestClassifier(n_estimators=20,verbose=False,n_jobs=-1)
dtc_model = DecisionTreeClassifier()
xgb_model=XGBClassifier()
knn_model = KNeighborsClassifier(n_neighbors=1)

In [26]:
def create_model_data(data,nam):
  mdf = data[["title","category"]]
  mdf = mdf[mdf['category']!="Unknown"].reset_index()
  mdf['cid'] = 0
  dict_a,_ = getuniqcateg(data)
  for index,i in enumerate(mdf.category):
    for j in dict_a.keys():
      if dict_a[j]==i:
        mdf.at[index,'cid'] = j
  v = CountVectorizer()
  words = v.fit_transform(mdf['title'].values)
  cats = mdf.cid.values
  filename = nam + '_vector.pkl'
  pickle.dump(v, open(filename, 'wb'))
  return words,cats
'''

This function create_model_data is designed to prepare data for training a machine learning model. It takes two parameters: data, 
which presumably contains information about titles and their corresponding categories, and nam, which is a string used to name the vectorizer file.


It extracts only the 'title' and 'category' columns from the input data.
It removes any rows where the category is labeled as "Unknown" and resets the index.
It assigns numerical category IDs to each category, making it easier for machine learning algorithms to process.
It creates a CountVectorizer object to convert the text data (titles) into numerical vectors.
It fits this vectorizer on the title data and transforms the titles into a matrix of word counts.
It stores the fitted vectorizer object into a pickle file with the name provided by the nam parameter.
It returns the word count matrix (words) and the numerical category IDs (cats) for further use in training the machine learning model


'''

'\n\nThis function create_model_data is designed to prepare data for training a machine learning model. It takes two parameters: data, \nwhich presumably contains information about titles and their corresponding categories, and nam, which is a string used to name the vectorizer file.\n\n\nIt extracts only the \'title\' and \'category\' columns from the input data.\nIt removes any rows where the category is labeled as "Unknown" and resets the index.\nIt assigns numerical category IDs to each category, making it easier for machine learning algorithms to process.\nIt creates a CountVectorizer object to convert the text data (titles) into numerical vectors.\nIt fits this vectorizer on the title data and transforms the titles into a matrix of word counts.\nIt stores the fitted vectorizer object into a pickle file with the name provided by the nam parameter.\nIt returns the word count matrix (words) and the numerical category IDs (cats) for further use in training the machine learning model\

In [27]:
def ensembleecreate(data,nam):
  print("-----------------Starting for",nam,"-----------------")
  X,y = create_model_data(data,nam)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
  dtc = dtc_model.fit(X,y)
  gnb = gnb_model.fit(X,y)
  xgb = dtc_model.fit(X,y)
  rfc = rfc_model.fit(X,y)

  knn = knn_model.fit(X_train,y_train)
  print("----------------------------------------","Implementing Ensemble model has started","----------------------------------------")
  estimators=[('gnb_model', gnb), ('xgb_model', xgb), ('dtc_model',dtc), ('knn_model', knn), ('rfc_model', rfc)]
  # ens_name = "ensemble_" + str(nam)
  ens_name = VotingClassifier(estimators, voting='hard')
  ens_name.fit(X,y)
  acc_ens = ens_name.score(X_test, y_test)
  print("----------------------------------------","Implementing Ensemble model has finished","----------------------------------------")
  print("\nThe Ensemble Model for %s has achieved an accuracy of %.4f" %(nam,acc_ens))
  filename = nam + '_model.pkl'
  pickle.dump(ens_name, open(filename, 'wb'))
  return ens_name  
'''
This Python function, ensembleecreate, is designed to create an ensemble model using a combination of different classifiers. Here's a brief explanation of what it does:

Input Parameters: The function takes two parameters: data, which represents the dataset, and nam, which is a string indicating the name of the model.
Model Training Data Preparation: Inside the function, create_model_data(data, nam) is called to prepare the feature matrix X and target vector y specific to the model indicated by nam.
Model Training: The function then splits the data into training and testing sets using train_test_split(). It trains individual classifiers (dtc_model, gnb_model, dtc_model, rfc_model, knn_model) on the entire dataset.
Ensemble Creation: An ensemble model (ens_name) is created using VotingClassifier from scikit-learn, which combines the predictions of the individual classifiers using majority voting (voting='hard').
Model Evaluation: The accuracy of the ensemble model is evaluated on the testing set using score() method.
Model Serialization: The trained ensemble model is saved to a file using pickle.dump() for future use.
Return: Finally, the function returns the trained ensemble model.

'''

"\nThis Python function, ensembleecreate, is designed to create an ensemble model using a combination of different classifiers. Here's a brief explanation of what it does:\n\nInput Parameters: The function takes two parameters: data, which represents the dataset, and nam, which is a string indicating the name of the model.\nModel Training Data Preparation: Inside the function, create_model_data(data, nam) is called to prepare the feature matrix X and target vector y specific to the model indicated by nam.\nModel Training: The function then splits the data into training and testing sets using train_test_split(). It trains individual classifiers (dtc_model, gnb_model, dtc_model, rfc_model, knn_model) on the entire dataset.\nEnsemble Creation: An ensemble model (ens_name) is created using VotingClassifier from scikit-learn, which combines the predictions of the individual classifiers using majority voting (voting='hard').\nModel Evaluation: The accuracy of the ensemble model is evaluated 

In [28]:
def ensembleperform():
  for i,j in condict.items():
    if j[j.category=="Unknown"].value_counts().sum() > 0:
      globals()[str("ensemble_") + str(i)]  = ensembleecreate(j,i)
ensembleperform()

-----------------Starting for CA -----------------
---------------------------------------- Implementing Ensemble model has started ----------------------------------------
---------------------------------------- Implementing Ensemble model has finished ----------------------------------------

The Ensemble Model for CA has achieved an accuracy of 0.9979
-----------------Starting for DE -----------------
---------------------------------------- Implementing Ensemble model has started ----------------------------------------
---------------------------------------- Implementing Ensemble model has finished ----------------------------------------

The Ensemble Model for DE has achieved an accuracy of 0.9978
-----------------Starting for FR -----------------
---------------------------------------- Implementing Ensemble model has started ----------------------------------------
---------------------------------------- Implementing Ensemble model has finished -----------------------------